In [93]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
from datetime import date
from datetime import timedelta
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import math 
import re
import pyodbc
from datetime import datetime
from sqlalchemy import create_engine, MetaData
import sqlalchemy as db
import urllib

pyodbc.drivers()
server = "120.120.120.145"
database = "Algo8"
username = "kamransultan"
password = "sul@888tan"

params = urllib.parse.quote_plus(
'DRIVER={ODBC Driver 17 for SQL Server};'+
'SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

#Definig connection 
try:      
    conx = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server}; SERVER=120.120.120.145; Database=Algo8; UID=kamransultan; PWD=sul@888tan;')
    print("Connection Successfully established!!")
except Exception as e:
    print(e)
    print('Connection not established')

Connection Successfully established!!


In [94]:
class TapeDemandCalculator:
    def __init__(self, engine):
        self.engine = engine
        self.conx = engine.connect()

    #Reading FabricRecipe
    def read_fabric_recipe(self):
        query = "SELECT * FROM [Algo8].[dbo].[FabricRecipe]"
        fab_recp = pd.read_sql(query, self.engine)
        return fab_recp
    
    
    def process_fabric_recipe2(self, fab_recp):
        # Filter Warp data
        df_warp = fab_recp[fab_recp['TapePropertyName'] == 'Warp'][['FabricId', 'TapeId', 'TapeName','NoOfTape_Auto','Master_Tape2Fabric_TapePercentage']]
        df_warp.columns = ['FabricId', 'warp_id', 'warp_name','warp_no_of_tape_auto','Warp%']
    
        # Filter Weft data
        df_weft = fab_recp[fab_recp['TapePropertyName'] == 'Weft'][['FabricId', 'TapeId', 'TapeName','NoOfTape_Auto','Master_Tape2Fabric_TapePercentage']]
        df_weft.columns = ['FabricId', 'weft_id', 'weft_name','weft_no_of_tape_auto','Weft%']
        
        df_warp_rf = fab_recp[fab_recp['TapePropertyName'] == 'Warp R/F'][['FabricId', 'TapeId', 'TapeName','NoOfTape_Auto','Master_Tape2Fabric_TapePercentage']]
        df_warp_rf.columns = ['FabricId', 'warp_rf_id',  'warp_rf_name','warp_rf_no_of_tape_auto','Warp_rf%']
    
        # Merge on FabricId
        df_ww = pd.merge(df_weft, df_warp, on='FabricId', how='left')
        df_ww = pd.merge(df_ww, df_warp_rf, on='FabricId', how='left')
        return df_ww
    
    def process_fabric_recipe(self, fab_recp):
    # Filter the rows for 'Warp' and 'Weft'
        df_filtered = fab_recp[fab_recp['TapePropertyName'].isin(['Warp', 'Weft','Warp R/F'])]

        # Remove duplicate entries if they exist
        df_filtered = df_filtered.drop_duplicates(subset=['FabricId', 'TapePropertyName'])

        # Pivot the DataFrame to transform 'TapePropertyName' into columns
        df_pivoted = df_filtered.pivot(index='FabricId', columns='TapePropertyName', values=['TapeId', 'TapeName','NoOfTape_Auto','Master_Tape2Fabric_TapePercentage'])

        # Reset index to flatten the pivot
        df_pivoted.reset_index(inplace=True)

        # Rename columns for clarity
        df_pivoted.columns = ['FabricId','warp_id', 'warp_rf_id','weft_id', 'weft_name', 'warp_rf_name','warp_name',
                              'warp_no_of_tape_auto', 'warp_rf_no_of_tape_auto','weft_no_of_tape_auto','Warp%','Warp_rf%','Weft%']
        df_pivoted['Warp_rf%'].fillna(0,inplace=True)
        df_pivoted['warp_rf_no_of_tape_auto'].fillna(0,inplace=True)
        return df_pivoted
    
    def read_weaving_planning(self):
        query = "SELECT [Previous_FabricId],[Previous_FabricCode],[FabricId], [FabricCode], [ULFabricBalanceToMake(Mtrs)], [Fabric Completion Date],[FabricTargetDate], [LoomNo], [Loom_Location],[LoomType],[Seq], [ChangeOver_start_date],[ChangeOverType],[ChangeOver_Start_time/Shift],[Production capacity per day(Mtrs)],[AuditDate] FROM [Algo8].[dbo].[Weaving_Planning]"
        df_demand = pd.read_sql(query, self.engine)
        return df_demand
    
    def runningFabrics(self,df_demand):
        df_running = df_demand.loc[df_demand['Seq']=='Running']
        df_demand = df_demand.loc[~(df_demand['Seq']=='Running')]
        return df_running,df_demand
    
    def freeze_fabrics(self,df_demand):
        query = "SELECT FabricId, LoomNo,FabricTargetDate FROM [Algo8].[dbo].[ChangeOver_Freeze_Form2] WHERE Freeze = 'True'"
        freeze_fabric = pd.read_sql(query, self.engine)
        freeze = freeze_fabric.merge(df_demand,on=['FabricId','LoomNo','FabricTargetDate'],how='inner')
        
        return freeze
         
    def final_demand(self,df_running,df_freeze):
        df_demand = pd.concat([df_running,df_freeze],axis=0)
        return df_demand
    
    def calculate_no_of_looms(self, df_demand):
        fabric_ids = list(df_demand['FabricId'])
        No_of_Looms = []
        for fabric_id in fabric_ids:
            filtered_df = df_demand.loc[(df_demand['FabricId'] == fabric_id)].copy()
            No_of_Looms.append(len(filtered_df['LoomNo'].unique()))
        df_demand['NoOfLooms'] = No_of_Looms

        return df_demand
    def read_master_fabric_data(self):
        query = "SELECT FabricId, FabricWidth, FabricGSM, FabricWarpMesh, FabricWeftMesh,FabricTypeMultiplyingFactor FROM [Algo8].[dbo].[Master_Fabric]"
        df_1_Master = pd.read_sql(query, self.engine)
        return df_1_Master

    #Read master tapes for properties of tape
    def read_master_tape_data(self,df):
        query = "SELECT [TapeId], [TapeDenier] FROM [Algo8].[dbo].[Master_Tape]"
        TapeDenier = pd.read_sql(query, self.engine)
        
        TapeDenier.rename(columns={'TapeId':'warp_id','TapeDenier':'Warp_Denier'},inplace=True)
        TapeDenier = TapeDenier.drop_duplicates()
        df = pd.merge(df, TapeDenier, on='warp_id', how='left')
        
        TapeDenier.rename(columns={'warp_id':'weft_id','Warp_Denier':'Weft_Denier'},inplace=True)
        TapeDenier = TapeDenier.drop_duplicates()
        df1 = pd.merge(df, TapeDenier, on='weft_id', how='left')
        
        TapeDenier.rename(columns={'weft_id':'warp_rf_id','Weft_Denier':'Warp_rf_Denier'},inplace=True)
        TapeDenier = TapeDenier.drop_duplicates()
        df2 = pd.merge(df1, TapeDenier, on='warp_rf_id', how='left')
        
        return df2
    
    def days_demand(self,df1,confg_days):
        
        days = confg_days['ConfiguredDays_Demand'].values[0]
        
        running_fabrics = df1[df1['ChangeOver_start_date'].isnull()]
        running_fabrics['ULFabricBalanceToMake(Mtrs)'] = running_fabrics.apply(
             lambda x: min(x['ULFabricBalanceToMake(Mtrs)'], x['Production capacity per day(Mtrs)'] *days), axis=1)
        
        upcoming_changeovers = df1[(df1['ChangeOver_start_date'].notnull()) & (df1['ChangeOver_start_date'] < (pd.Timestamp.now() + pd.Timedelta(days=days)))]
        upcoming_changeovers['ULFabricBalanceToMake(Mtrs)'] = upcoming_changeovers.apply(
            lambda x: min(x['ULFabricBalanceToMake(Mtrs)'], x['Production capacity per day(Mtrs)'] *days), axis=1)
        
        df_new= pd.concat([running_fabrics,upcoming_changeovers])
        
        return df_new

        
        
    #Calculation of Fabric Widht, Weft demand and warp demand   
    def calculate_tape_demand2(self, df):
        df["FabricWidth"] = df["FabricWidth"] / 100
        df["FabricInKGs"] = np.round((df["FabricWidth"] * df["ULFabricBalanceToMake(Mtrs)"] * df["FabricGSM"]*df['FabricTypeMultiplyingFactor']) / 1000, 2)

        
        df['Warp_Tape'] = 0.0
        df['Warp_rf_Tape'] = 0.0
        df['Weft_Tape'] = 0.0

        # Iterate through the DataFrame and calculate 'Warp_Tape' iteratively
        for index, row in df.iterrows():
            df.at[index, 'Warp_Tape'] = round((row['FabricInKGs'] * row['Warp%']) / 100, 2)
            df.at[index, 'Warp_rf_Tape'] = round((row['FabricInKGs'] * row['Warp_rf%']) / 100, 2)
            df.at[index, 'Weft_Tape'] = round((row['FabricInKGs'] * row['Weft%']) / 100, 2)
        

        df["Tape_Target_Date"] = df['ChangeOver_start_date'] - pd.DateOffset(days=1) 
        
        df['Tape_Target_Date'] = pd.to_datetime(df['Tape_Target_Date'])
        df['Demand_Source'] = 'Weaving_Planning'
        
        dfk = df[['Previous_FabricId','Previous_FabricCode','FabricId', 'FabricCode', 'ChangeOver_start_date', 'Tape_Target_Date',
                   'ULFabricBalanceToMake(Mtrs)','ChangeOverType','Production capacity per day(Mtrs)', 'LoomNo','Loom_Location','LoomType','NoOfLooms', 
                   'Seq','warp_id', 'warp_name','warp_rf_id', 'warp_rf_name', 'weft_id', 'weft_name',
                    'Warp_Tape','Warp_rf_Tape', 'Weft_Tape',  
                   'FabricInKGs','FabricTypeMultiplyingFactor','Weft%','Warp%','Warp_rf%','PreLogic_FinalFactor',
                   'warp_no_of_tape_auto', 'warp_rf_no_of_tape_auto','Demand_Source']]
        
        
        return dfk

    
        
    def calculate_load_unload(self, dfk):
        
        dfk['prev_warp_rf_id'].fillna(0,inplace=True)
        dfk['warp_rf_id'].fillna(0,inplace=True)
        dfk['Prev_Warp_Unload'] = 0
        dfk['Prev_Warp_rf_Unload'] = 0
        dfk['Weft_Load'] = 0
        dfk['Warp_Load'] = 0
        dfk['Warp_rf_Load'] = 0
        dfk['Weft_Unload'] = 0
        dfk['Warp_Unload'] = 0
        dfk['Warp_rf_Unload'] = 0
        dfk['Warp_Change'] = ' '
        dfk['Warp_rf_Change'] = ' '
        dfk['Warp_Tape_Diff'] = 0
        dfk['Warp_rf_Tape_Diff'] = 0

        for index, row in dfk.iterrows():
            if row['warp_id'] != row['prev_warp_id']  :
                dfk.loc[index,'Warp_Change'] = 'Yes'
                dfk.loc[index, 'Warp_Load'] = row['warp_no_of_tape_auto'] * 1.5 * row['PreLogic_FinalFactor']
                dfk.loc[index, 'Warp_Unload'] = row['warp_no_of_tape_auto'] * 0.6 * row['PreLogic_FinalFactor']
                dfk.loc[index,'Prev_Warp_Unload'] =  row['prev_warp_no_of_tape_auto'] * 0.6 * row['prev_PreLogic_FinalFactor']
            else:
                dfk.loc[index,'Warp_Change'] = 'No'
                dfk.loc[index,'Warp_Tape_Diff']= row['warp_no_of_tape_auto'] - row['prev_warp_no_of_tape_auto']
                if row['warp_no_of_tape_auto'] > row['prev_warp_no_of_tape_auto']:  # increase in no of tapes
                    
                    dfk.loc[index, 'Warp_Load'] = (row['warp_no_of_tape_auto'] - row['prev_warp_no_of_tape_auto']) * 1.5 * row['PreLogic_FinalFactor']
                    dfk.loc[index,'Warp_Unload'] = row['warp_no_of_tape_auto'] * 0.6 * row['PreLogic_FinalFactor']
                    dfk.loc[index, 'Prev_Warp_Unload'] = 0
                elif row['warp_no_of_tape_auto'] < row['prev_warp_no_of_tape_auto']:  # decrease in no of tapes
                    
                    dfk.loc[index, 'Warp_Load'] = 0
                    dfk.loc[index, 'Warp_Unload'] = row['warp_no_of_tape_auto'] * 0.6 * row['PreLogic_FinalFactor']
                    dfk.loc[index, 'Prev_Warp_Unload'] = (row['prev_warp_no_of_tape_auto'] - row['warp_no_of_tape_auto']) * 0.6 * row['prev_PreLogic_FinalFactor']
                else :#no increase decrease
                    dfk.loc[index, 'Warp_Load'] = 0
                    dfk.loc[index, 'Warp_Unload'] =0
                    dfk.loc[index, 'Prev_Warp_Unload'] = row['prev_warp_no_of_tape_auto'] * 0.6 * row['prev_PreLogic_FinalFactor']
                
            if row['warp_rf_id'] != row['prev_warp_rf_id']:
                dfk.loc[index,'Warp_rf_Change'] = 'Yes'
                dfk.loc[index, 'Warp_rf_Load'] = row['warp_rf_no_of_tape_auto'] * 1.5 * row['PreLogic_FinalFactor']
                dfk.loc[index, 'Warp_rf_Unload'] = row['warp_rf_no_of_tape_auto'] * 0.6 * row['PreLogic_FinalFactor']
                dfk.loc[index,'Prev_Warp_rf_Unload'] =  row['prev_warp_rf_no_of_tape_auto'] * 0.6 * row['prev_PreLogic_FinalFactor']
            else:
                dfk.loc[index,'Warp_rf_Change'] = 'No'
                dfk.loc[index,'Warp_rf_Tape_Diff'] = row['warp_rf_no_of_tape_auto'] - row['prev_warp_rf_no_of_tape_auto']
                if row['warp_rf_no_of_tape_auto'] > row['prev_warp_rf_no_of_tape_auto']:  # increase in no of tapes
                    dfk.loc[index, 'Warp_rf_Load'] = (row['warp_rf_no_of_tape_auto'] - row['prev_warp_rf_no_of_tape_auto']) * 1.5 * row['PreLogic_FinalFactor']
                    dfk.loc[index,'Warp_rf_Unload'] = row['warp_rf_no_of_tape_auto'] * 0.6 * row['PreLogic_FinalFactor']
                    dfk.loc[index, 'Prev_Warp_rf_Unload'] = 0
                elif row['warp_rf_no_of_tape_auto'] < row['prev_warp_rf_no_of_tape_auto'] :  # decrease in no of tapes
                    dfk.loc[index, 'Warp_rf_Load'] = 0
                    dfk.loc[index, 'Warp_rf_Unload'] = row['warp_rf_no_of_tape_auto'] * 0.6 * row['PreLogic_FinalFactor']
                    dfk.loc[index, 'Prev_Warp_rf_Unload'] = (row['prev_warp_rf_no_of_tape_auto'] - row['warp_rf_no_of_tape_auto']) * 0.6 * row['prev_PreLogic_FinalFactor']
                else : #no increase decrease
                    dfk.loc[index, 'Warp_rf_Load'] = 0
                    dfk.loc[index, 'Warp_rf_Unload'] = 0
                    dfk.loc[index, 'Prev_Warp_rf_Unload'] = row['prev_warp_rf_no_of_tape_auto'] * 0.6 * row['prev_PreLogic_FinalFactor']
        return dfk
                
    def total_demand(self,df):
        df['Total_Warp_Demand'] = df['Warp_Tape'] + df['Warp_Load']
        df['Total_Weft_Demand'] = df['Weft_Tape']
        df['Total_Warp_rf_Demand'] = df['Warp_rf_Tape'] + df['Warp_rf_Load']
        return df
    
    def invent_df(self,df_invent_GR,df_invent_D19,demand_df):
        demand_df['Warp_Inventory_Utilised'] = 0
        demand_df['Weft_Inventory_Utilised'] = 0
        demand_df['Warp_rf_Inventory_Utilised'] = 0
        for index, row in demand_df.iterrows():
            weft_id = row['weft_id']
            warp_id = row['warp_id']
            warp_rf_id = row['warp_rf_id']
            weft_demand = row['Total_Weft_Demand']
            warp_demand = row['Total_Warp_Demand']
            warp_rf_demand = row['Total_Warp_rf_Demand']
            
            if row['Loom_Location'] =='Gajner Road':
                
                if warp_id in df_invent_GR['PlantProd_TapeId'].values:
                    
                    
                    inventory = df_invent_GR.loc[df_invent_GR['PlantProd_TapeId'] == warp_id, 'Total'].values[0]
                    
                    i = df_invent_GR.loc[df_invent_GR['PlantProd_TapeId'] == warp_id].index[0]
                    
                    if warp_demand >= inventory :
                        demand_df.loc[index, 'Total_Warp_Demand'] = warp_demand - inventory
                        demand_df.loc[index,'Warp_Inventory_Utilised'] = inventory
                        df_invent_GR.loc[i, 'Total'] = 0
                    else :
                        demand_df.loc[index, 'Total_Warp_Demand'] = 0
                        demand_df.loc[index,'Warp_Inventory_Utilised'] = warp_demand
                        df_invent_GR.loc[i, 'Total'] = inventory- warp_demand
                    
                if weft_id in df_invent_GR['PlantProd_TapeId'].values:
                    inventory = df_invent_GR.loc[df_invent_GR['PlantProd_TapeId'] == weft_id, 'Total'].values[0]
                    i =df_invent_GR.loc[df_invent_GR['PlantProd_TapeId'] == weft_id].index[0]
                    if weft_demand >= inventory :
                        demand_df.loc[index, 'Total_Weft_Demand'] = weft_demand - inventory
                        demand_df.loc[index,'Weft_Inventory_Utilised'] = inventory
                        df_invent_GR.loc[i, 'Total'] = 0
                    else :
                        demand_df.loc[index, 'Total_Weft_Demand'] = 0 
                        demand_df.loc[index,'Weft_Inventory_Utilised'] = weft_demand
                        df_invent_GR.loc[i, 'Total'] = inventory - weft_demand

                if warp_rf_id in df_invent_GR['PlantProd_TapeId'].values:
                
                    inventory = df_invent_GR.loc[df_invent_GR['PlantProd_TapeId'] == warp_rf_id, 'Total'].values[0]
                    i = df_invent_GR.loc[df_invent_GR['PlantProd_TapeId'] == warp_rf_id].index[0]
                    
                    if warp_rf_demand >= inventory :
                        demand_df.loc[index, 'Total_Warp_rf_Demand'] = warp_rf_demand - inventory
                        demand_df.loc[index,'Warp_rf_Inventory_Utilised'] = inventory
                        df_invent_GR.loc[i, 'Total'] = 0
                    else :
                        
                        demand_df.loc[index, 'Total_Warp_rf_Demand'] = 0  
                        demand_df.loc[index,'Warp_rf_Inventory_Utilised'] = warp_rf_demand
                        df_invent_GR.loc[i, 'Total'] = inventory- warp_rf_demand
                    
                        
            elif row['Loom_Location'] =='D-19':
                
                if warp_id in df_invent_D19['PlantProd_TapeId'].values:
                
                    inventory = df_invent_D19.loc[df_invent_D19['PlantProd_TapeId'] == warp_id, 'Total'].values[0]
                    
                    i = df_invent_D19.loc[df_invent_D19['PlantProd_TapeId'] == warp_id].index[0]
                    
                    if warp_demand >= inventory :
                        demand_df.loc[index, 'Total_Warp_Demand'] = warp_demand - inventory
                        demand_df.loc[index,'Warp_Inventory_Utilised'] = inventory
                        df_invent_D19.loc[i, 'Total'] = 0
                    else :
                        demand_df.loc[index, 'Total_Warp_Demand'] = 0
                        demand_df.loc[index,'Warp_Inventory_Utilised'] = warp_demand
                        df_invent_D19.loc[i, 'Total'] = inventory- warp_demand
                    
                if weft_id in df_invent_D19['PlantProd_TapeId'].values:
                    inventory = df_invent_D19.loc[df_invent_D19['PlantProd_TapeId'] == weft_id, 'Total'].values[0]
                    i =df_invent_D19.loc[df_invent_D19['PlantProd_TapeId'] == weft_id].index[0]
                    if weft_demand >= inventory :
                        demand_df.loc[index, 'Total_Weft_Demand'] = weft_demand - inventory
                        demand_df.loc[index,'Weft_Inventory_Utilised'] = inventory
                        df_invent_D19.loc[i, 'Total'] = 0
                    else :
                        demand_df.loc[index, 'Total_Weft_Demand'] = 0
                        demand_df.loc[index,'Weft_Inventory_Utilised'] = weft_demand 
                        df_invent_D19.loc[i, 'Total'] = inventory - weft_demand

                if warp_rf_id in df_invent_D19['PlantProd_TapeId'].values:
                    
                    inventory = df_invent_D19.loc[df_invent_D19['PlantProd_TapeId'] == warp_rf_id, 'Total'].values[0]
                    i = df_invent_D19.loc[df_invent_D19['PlantProd_TapeId'] == warp_rf_id].index[0]
                    
                    if warp_rf_demand >= inventory :
                        demand_df.loc[index, 'Total_Warp_rf_Demand'] = warp_rf_demand - inventory
                        demand_df.loc[index,'Warp_rf_Inventory_Utilised'] = inventory
                        df_invent_D19.loc[i, 'Total'] = 0
                    else :
                        
                        demand_df.loc[index, 'Total_Warp_rf_Demand'] = 0 
                        demand_df.loc[index,'Warp_rf_Inventory_Utilised'] = warp_rf_demand 
                        df_invent_D19.loc[i, 'Total'] = inventory- warp_rf_demand

        return demand_df   
   
                
                 
    def calculate_demand_df(self,demand_df):
        print("Demand_df_size :",demand_df.shape)

        weft_demand_df = demand_df[['FabricId','weft_id','weft_name','Loom_Location','LoomNo',
                                    'ChangeOver_start_date','Tape_Target_Date','Total_Weft_Demand','Weft_Inventory_Utilised',
                                    'LoomType','Weft_Tape','Weft_Load','Weft_Unload','Seq',
                                    
                                    'ULFabricBalanceToMake(Mtrs)','Production capacity per day(Mtrs)','PreLogic_FinalFactor',
                                    'Initial_Weft_Demand','Demand_Source']]
        
        weft_demand_df['TapePropertyName'] = 'Weft'
        weft_demand_df['Weft_Demand']= weft_demand_df['Total_Weft_Demand']
        weft_demand_df['Warp_Demand']= 0
        weft_demand_df['Warp_rf_Demand']= 0
        weft_demand_df.rename(columns={'weft_id': 'TapeId', 'weft_name': 'Tape_name','Total_Weft_Demand':'Total_Demand',
                                       'Initial_Weft_Demand':'Initial_Demand','Weft_Tape':'Tape(Kg)','Weft_Load':'Tape_Load',
                                       'Weft_Unload':'Tape_Unload','Weft_Inventory_Utilised':'Inventory_Utilised'},inplace=True)
        
        
        warp_demand_df = demand_df[['FabricId','warp_id','warp_name','Loom_Location','LoomNo',
                                    'ChangeOver_start_date', 'Tape_Target_Date','Total_Warp_Demand','Warp_Inventory_Utilised',
                                    'LoomType','Warp_Tape','Warp_Load','Warp_Unload','Seq',
                                    
                                    'ULFabricBalanceToMake(Mtrs)','Production capacity per day(Mtrs)','PreLogic_FinalFactor',
                                    'Initial_Warp_Demand','Demand_Source']]
        
        warp_demand_df['TapePropertyName'] = 'Warp'
        warp_demand_df['Weft_Demand']= 0
        warp_demand_df['Warp_Demand']= warp_demand_df['Total_Warp_Demand']
        warp_demand_df['Warp_rf_Demand']= 0
        warp_demand_df.rename(columns={'warp_id': 'TapeId',  'warp_name': 'Tape_name','Total_Warp_Demand':'Total_Demand',
                                       'Initial_Warp_Demand':'Initial_Demand','Warp_Tape':'Tape(Kg)','Warp_Load':'Tape_Load',
                                       'Warp_Unload':'Tape_Unload','Warp_Inventory_Utilised':'Inventory_Utilised'}, inplace=True)
        
        warp_rf_demand_df = demand_df[['FabricId','warp_rf_id','warp_rf_name','Loom_Location','LoomNo',
                                    'ChangeOver_start_date', 'Tape_Target_Date','Total_Warp_rf_Demand','Warp_rf_Inventory_Utilised',
                                    'LoomType','Warp_rf_Tape','Warp_rf_Load','Warp_rf_Unload','Seq',
                                    
                                    'ULFabricBalanceToMake(Mtrs)','Production capacity per day(Mtrs)','PreLogic_FinalFactor',
                                    'Initial_Warp_rf_Demand','Demand_Source']]
        
        warp_rf_demand_df['TapePropertyName'] = 'Warp_rf'
        warp_rf_demand_df['Weft_Demand']= 0
        warp_rf_demand_df['Warp_Demand']= 0
        warp_rf_demand_df['Warp_rf_Demand']= warp_rf_demand_df['Total_Warp_rf_Demand']
        
        warp_rf_demand_df= warp_rf_demand_df[warp_rf_demand_df['warp_rf_id']!=0]
        
        warp_rf_demand_df.rename(columns={'warp_rf_id': 'TapeId',  'warp_rf_name': 'Tape_name','Total_Warp_rf_Demand':'Total_Demand',
                                          'Initial_Warp_rf_Demand':'Initial_Demand','Warp_rf_Tape':'Tape(Kg)','Warp_rf_Load':'Tape_Load',
                                       'Warp_rf_Unload':'Tape_Unload','Warp_rf_Inventory_Utilised':'Inventory_Utilised'}, inplace=True)
        
        
        
        
        print("Weft_size: ",weft_demand_df.shape)
        print("Warp_size: ",warp_demand_df.shape)
        print("Warp_rf_size: ",warp_rf_demand_df.shape)

        tape_demand_df = pd.concat([weft_demand_df,warp_demand_df,warp_rf_demand_df],ignore_index=True)
        
        tape_demand_df.to_excel("merged_demand_31_10.xlsx",index=False)
        tape_demand = tape_demand_df.groupby(['TapeId','FabricId','Tape_Target_Date','Loom_Location','LoomNo','LoomType']).agg({
            'Tape_name':'first',
            'TapePropertyName':'unique',
            'Seq': 'first',
            'Total_Demand':'sum',
            'Initial_Demand':'sum',
            'Inventory_Utilised':'sum',
            'Tape(Kg)':'sum',
            'Weft_Demand':'sum',
            'Warp_Demand':'sum',
            'Warp_rf_Demand': 'sum',
            'Tape_Load' : 'sum',
            'Tape_Unload' : 'sum',
            'Production capacity per day(Mtrs)':'first',
            'ULFabricBalanceToMake(Mtrs)':'first',
            'PreLogic_FinalFactor' :'first',
            'Demand_Source' :'first'
        }).reset_index()
        
        print("Grouped_Tape_demand_size: ",tape_demand.shape)
        
        
        return tape_demand
    
    
   
    def read_master_tape_data_extended(self):
        query = "SELECT TapeId, TapeWidth, TapeColour, TapeDenier, TapeMarking FROM [Algo8].[dbo].[Master_Tape]"
        dfkk = pd.read_sql(query, self.engine)
        return dfkk

    #Read Master Fabric
    def read_master_fabric_data_extended(self):
        query = "SELECT FabricId, FabricWarpMesh FROM [Algo8].[dbo].[Master_Fabric]"
        dfk = pd.read_sql(query, self.engine)
        return dfk

    #Read Master tape recipe
    def read_master_tape_recipe(self):
        #query = "SELECT Master_TapeRecipe_TapeId, Master_TapeRecipe_TapeMaterialId, Master_TapeRecipe_Percentage FROM [Algo8].[dbo].[Master_TapeRecipe]"
        query = "SELECT TapeId, MaterialId, Master_TapeRecipe_Percentage FROM [Algo8].[dbo].[Master_TapeRecipe]"
        dfkkk = pd.read_sql(query, self.engine)
        dfkkk = dfkkk.loc[dfkkk.groupby('TapeId')['Master_TapeRecipe_Percentage'].idxmax()]
        dfkkk = dfkkk.drop(['Master_TapeRecipe_Percentage'], axis=1)
        #dfkkk = dfkkk.rename(columns={'Master_TapeRecipe_TapeId': 'TapeId', 'Master_TapeRecipe_TapeMaterialId': 'MaterialId'})
        return dfkkk
    
    def process_demand_df2(self,demand_df,dfkk,dfk):
        
        demand_df = pd.merge(demand_df, dfkk, on='TapeId')
        demand_df = pd.merge(demand_df, dfk, on='FabricId', how='left')

        #Extract the exact tape color name.  
        Tape_colour = demand_df['TapeColour']
        Tape_colour_updated = []
        for colour in Tape_colour:
            colour = colour.lower()
            if 'white' in colour:
                Tape_colour_updated.append('white')
            else:
                match = re.search(r'([a-zA-Z\s]+)', colour)
                if match:
                    color_name = match.group(1).strip()
                    Tape_colour_updated.append(color_name)

        demand_df['TapeColour'] = Tape_colour_updated

        #Defining the Denier to be low if it is <1000 else high
        Tape_Denier = []
        TapeDenier = demand_df['TapeDenier']
        for denier in TapeDenier:
            if denier <= 1000:
                Tape_Denier.append('Low')
            else:
                Tape_Denier.append('High')
        demand_df['Tape_Denier'] = Tape_Denier

        #Defing the width to narrow, standard and wider
        Tape_width = []
        TapeWidth = demand_df['TapeWidth']
        demand_df['Tape_Width'] = demand_df['TapeWidth'].round(2)
        for width in TapeWidth:
            if width < 2.5:
                Tape_width.append('Narrow')
            elif 2.5 <= width <= 3.1:
                Tape_width.append('Standard')
            else:
                Tape_width.append('Wider')
        demand_df['TapeWidth'] = Tape_width

        demand_df = demand_df.sort_values(['Tape_Target_Date', 'TapeDenier'])

        #Extracting the value  Tape UV
        TapeUV = []
        for index, row in demand_df.iterrows():
            tapename = row['Tape_name']
            if tapename == 0:
                TapeUV.append(0)
                continue
            else :
                pattern = r"NON UV"
                match = re.search(pattern, tapename, re.IGNORECASE)
                if match:
                    TapeUV.append(0)
                else:
                    TapeUV.append(1)
        demand_df['TapeUv'] = TapeUV

        return demand_df
        
    

In [95]:
calculator = TapeDemandCalculator(engine)

In [96]:
fab_recp = calculator.read_fabric_recipe()

df_ww = calculator.process_fabric_recipe2(fab_recp)
#df_ww.to_excel("fabric_recipe_20_10.xlsx",index='False')

df_demand = calculator.read_weaving_planning() 

df_running , df_demand = calculator.runningFabrics(df_demand)

df_freeze = calculator.freeze_fabrics(df_demand)
df_demand = calculator.final_demand(df_running,df_freeze)

df_demand.to_excel("demand_intial.xlsx",index=False)


In [97]:

#daywise no of changeovers
df_change = df_demand.loc[df_demand['Seq']!='Running']
df_change = df_change.loc[df_change['ChangeOverType']!='No ChangeOver']

df_change['ChangeOver_start_date'] = pd.to_datetime(df_change['ChangeOver_start_date'])
df_change = df_change[(df_change['ChangeOver_start_date'] < (pd.Timestamp.now() + pd.Timedelta(days=3)))]
df_change['ChangeOver_start_date'] = df_change['ChangeOver_start_date'].dt.date

day_wise_changes = df_change.groupby('ChangeOver_start_date')['ChangeOverType'].value_counts().reset_index()

day_wise_changes = day_wise_changes.groupby('ChangeOver_start_date')['count'].sum().reset_index()
day_wise_changes.rename(columns={'count':'No_of_changeovers'},inplace=True)

day_wise_changes.to_excel('day_wise_changeovers.xlsx',index=False)

In [98]:
df_demand.columns

Index(['Previous_FabricId', 'Previous_FabricCode', 'FabricId', 'FabricCode',
       'ULFabricBalanceToMake(Mtrs)', 'Fabric Completion Date',
       'FabricTargetDate', 'LoomNo', 'Loom_Location', 'LoomType', 'Seq',
       'ChangeOver_start_date', 'ChangeOverType',
       'ChangeOver_Start_time/Shift', 'Production capacity per day(Mtrs)',
       'AuditDate'],
      dtype='object')

In [99]:
df_demand.to_excel("demand.xlsx",index=False)

In [100]:
df_demand.drop(columns={'FabricTargetDate'},inplace=True)

In [101]:

#running fabric
#df_demand = df_demand.loc[df_demand['Seq']=='Running']

#changeover date filters
df_demand['ChangeOver_start_date'].fillna(df_demand['AuditDate'], inplace=True)
df_demand['ChangeOver_start_date'] = pd.to_datetime(df_demand['ChangeOver_start_date'])
df_demand = df_demand[~(df_demand["AuditDate"]>= df_demand["Fabric Completion Date"])]  

#adding no of layers 
query="SELECT FabricId, LoomNo, PreLogic_FinalFactor FROM [Algo8].[dbo].[ExclusionCriteria_AllFabrics]"
df_layers = pd.read_sql(query, engine)
df_demand = pd.merge(df_demand, df_layers, on=['FabricId','LoomNo'],how='left')

#adding loom location
query = "SELECT * FROM [Algo8].[dbo].[LoomNo_and_Location]"
loom_location = pd.read_sql(query, engine)


df_demand.loc[df_demand['Seq'] == 'Running', 'Loom_Location'] = df_demand.loc[df_demand['Seq'] == 'Running', 'LoomNo'].map(loom_location.set_index('LoomNo')['Loom_Location'])
#df_demand.to_excel("weaving_planning_20_10.xlsx",index=False)

df_demand = calculator.calculate_no_of_looms(df_demand)

df_demand=pd.merge(df_demand,df_ww,on=['FabricId'],how='left')
#df_demand.to_excel("demand_df_20_10.xlsx",index=False)

df_1_Master = calculator.read_master_fabric_data()

df = pd.merge(df_demand,df_1_Master,on="FabricId",how='left')

df1 = calculator.read_master_tape_data(df)

#df1.to_excel("Demand_before_14_11.xlsx",index=False)


#configured days demand
query = 'SELECT ConfiguredDays_Demand FROM [Algo8].[dbo].[Tapeline_ConfiguredDays_Demand_Schedule]'
confg_days = pd.read_sql(query, engine)
df_new = calculator.days_demand(df1,confg_days) 

#calculating weft warp demand
dfk = calculator.calculate_tape_demand2(df_new)


#adding prev tape layers
prev_layers = df_layers.copy()
prev_layers.rename(columns = {'FabricId':'Previous_FabricId'},inplace=True)
dfk = pd.merge(dfk,prev_layers,on=['Previous_FabricId','LoomNo'],how='left')

dfk.rename(columns={'PreLogic_FinalFactor_y':'prev_PreLogic_FinalFactor'},inplace=True)
dfk.rename(columns={'PreLogic_FinalFactor_x':'PreLogic_FinalFactor'},inplace=True)

fab_recipe = df_ww.copy()
fab_recipe.rename(columns={'FabricId':'Previous_FabricId'},inplace=True)
dfk2 = pd.merge(dfk,fab_recipe,on='Previous_FabricId',how ='left')


dfk2.rename(columns={'warp_id_x':'warp_id','warp_name_x':'warp_name','warp_rf_id_x':'warp_rf_id',
                     'warp_rf_name_x':'warp_rf_name','weft_id_x':'weft_id','weft_name_x':'weft_name',
                     'Weft%_x':'Weft%','Warp%_x':'Warp%','Warp_rf%_x':'Warp_rf%',
                     'warp_no_of_tape_auto_x':'warp_no_of_tape_auto','warp_rf_no_of_tape_auto_x':'warp_rf_no_of_tape_auto'},inplace=True)

dfk2.rename(columns={'warp_id_y':'prev_warp_id','warp_name_y':'prev_warp_name','warp_rf_id_y':'prev_warp_rf_id',
                     'warp_rf_name_y':'prev_warp_rf_name','weft_id_y':'prev_weft_id','weft_name_y':'prev_weft_name',
                     'Weft%_y':'prev_weft%','Warp%_y':'prev_warp%','Warp_rf%_y':'prev_warp_rf%',
                     'warp_no_of_tape_auto_y':'prev_warp_no_of_tape_auto','warp_rf_no_of_tape_auto_y':'prev_warp_rf_no_of_tape_auto'},inplace=True)

#calculation of demand load and unload
demand_load_unload = calculator.calculate_load_unload(dfk2)
demand_load_unload = calculator.total_demand(demand_load_unload)



#adding tape plant stock demands
query = 'SELECT PlantProd_TapeId, Total FROM [Algo8].[dbo].[TapePlantStock_Gajner]'
df_invent_GR = pd.read_sql(query, engine)
df_invent_GR = df_invent_GR[df_invent_GR['Total'] > 0 ]


query = 'SELECT PlantProd_TapeId, Total FROM [Algo8].[dbo].[TapePlantStock_D19]'
df_invent_D19 = pd.read_sql(query, engine)
df_invent_D19 = df_invent_D19[df_invent_D19['Total'] > 0 ]


demand_df = demand_load_unload.copy()
demand_df['Initial_Weft_Demand'] = demand_df['Total_Weft_Demand']
demand_df['Initial_Warp_Demand'] = demand_df['Total_Warp_Demand']
demand_df['Initial_Warp_rf_Demand'] = demand_df['Total_Warp_rf_Demand'] 

#applyling inventory rules
demand_in = calculator.invent_df(df_invent_GR,df_invent_D19,demand_df)

demand_df2 = calculator.calculate_demand_df(demand_in)




Demand_df_size : (198, 66)
Weft_size:  (198, 23)
Warp_size:  (198, 23)
Warp_rf_size:  (33, 23)
Grouped_Tape_demand_size:  (311, 22)


In [102]:
demand_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   TapeId                             311 non-null    float64       
 1   FabricId                           311 non-null    int64         
 2   Tape_Target_Date                   311 non-null    datetime64[ns]
 3   Loom_Location                      311 non-null    object        
 4   LoomNo                             311 non-null    int64         
 5   LoomType                           311 non-null    object        
 6   Tape_name                          311 non-null    object        
 7   TapePropertyName                   311 non-null    object        
 8   Seq                                311 non-null    object        
 9   Total_Demand                       311 non-null    float64       
 10  Initial_Demand                     311

In [103]:
#adding Buffer plan load
query = 'SELECT * FROM [Algo8].[dbo].[Tapeline_BufferPlan]'
buffer_demand = pd.read_sql(query,engine)
buffer_demand.rename(columns={'Qty(Kg)':'Total_Demand','TargetDate':'Tape_Target_Date'},inplace=True)
buffer_demand['Demand_Source'] = 'Buffer_demand'
buffer_demand['Loom_Location'] = 'Gajner Road'

query = 'SELECT TapeDescription, TapeId FROM [Algo8].[dbo].[Master_TapeRecipe]'
tape_desc = pd.read_sql(query,engine)

tape_desc['TapeId'] = pd.to_numeric(tape_desc['TapeId'], errors='coerce')



In [104]:
buffer_demand.shape

(27, 8)

In [105]:
buffer_demand = pd.merge(buffer_demand,tape_desc,on='TapeId',how='left')

In [116]:
buffer_demand.drop_duplicates(inplace=True)

In [117]:
buffer_demand.shape

(27, 6)

In [118]:
buffer_demand.rename(columns={'TapeDescription':'Tape_name'},inplace=True)

buffer_demand = buffer_demand[['TapeId','Tape_name','Tape_Target_Date','Total_Demand','Demand_Source','Loom_Location']]
buffer_demand = buffer_demand[buffer_demand['Tape_Target_Date']<= (pd.Timestamp.now() + pd.Timedelta(days=confg_days['ConfiguredDays_Demand'].values[0]))]
demand_df2= pd.concat([demand_df2,buffer_demand])

In [132]:
buffer_demand

,TapeId,Tape_name,Tape_Target_Date,Total_Demand,Demand_Source,Loom_Location
0,670,NaN,2024-01-16,3500.0,Buffer_demand,Gajner Road
1,1225,PP-Wider Tape (Webbing),2024-01-16,3500.0,Buffer_demand,Gajner Road
5,1224,PP-Wider Tape (Webbing),2024-01-16,500.0,Buffer_demand,Gajner Road
10,1227,PP-Wider Tape (Webbing),2024-01-16,250.0,Buffer_demand,Gajner Road
15,1226,PP-Wider Tape (Webbing),2024-01-16,800.0,Buffer_demand,Gajner Road
20,1229,PP-Wider Tape (Webbing),2024-01-16,250.0,Buffer_demand,Gajner Road
25,419,PP-Narrow Tape,2024-01-16,3500.0,Buffer_demand,Gajner Road
30,883,PP-Fabrilated Tape,2024-01-16,2400.0,Buffer_demand,Gajner Road
35,667,PP-Wider Tape,2024-01-16,1000.0,Buffer_demand,Gajner Road
40,670,NaN,2024-01-17,3500.0,Buffer_demand,Gajner Road


In [120]:
demand_df2['Demand_Source'].unique()

array(['Weaving_Planning', 'Buffer_demand'], dtype=object)

In [124]:
#adding tape specs
dfkk = calculator.read_master_tape_data_extended()
dfk = calculator.read_master_fabric_data_extended()
dfkkk = calculator.read_master_tape_recipe()
demand_df3 = pd.merge(demand_df2, dfkkk, on='TapeId')


#filler values
query ="SELECT TapeId , Master_TapeRecipe_Percentage, Master_TapeRecipe_ApplicableFrom, MaterialName, IsRP FROM [Algo8].[dbo].[Master_TapeRecipe1] where MaterialName like '%Filler%'"
df_filler = pd.read_sql(query, engine)

df_filler.dropna(inplace=True)
df_filler['Master_TapeRecipe_ApplicableFrom'] = pd.to_datetime(df_filler['Master_TapeRecipe_ApplicableFrom'])

result = df_filler.sort_values('Master_TapeRecipe_ApplicableFrom', ascending=False).drop_duplicates('TapeId')[['TapeId', 'Master_TapeRecipe_Percentage']]
result.rename(columns={'Master_TapeRecipe_Percentage':'TapeFiller'},inplace=True)
demand_df3 = pd.merge(demand_df3,result,on='TapeId',how='left')

#RP values
query ="SELECT TapeId , Master_TapeRecipe_Percentage, Master_TapeRecipe_ApplicableFrom, MaterialName, IsRP FROM [Algo8].[dbo].[Master_TapeRecipe1] where MaterialName like '%REP%'"
df_RP = pd.read_sql(query, engine)

df_RP.dropna(inplace=True)
df_RP['Master_TapeRecipe_ApplicableFrom'] = pd.to_datetime(df_RP['Master_TapeRecipe_ApplicableFrom'])

result = df_RP.sort_values('Master_TapeRecipe_ApplicableFrom', ascending=False).drop_duplicates('TapeId')[['TapeId', 'Master_TapeRecipe_Percentage']]
result.rename(columns={'Master_TapeRecipe_Percentage':'TapeRP'},inplace=True)
demand_df3 = pd.merge(demand_df3,result,on='TapeId',how='left')


demand_df3.fillna(0,inplace=True)


#extracting color,  tape name
demand_df3 = calculator.process_demand_df2(demand_df3, dfkk, dfk)
demand_df3['RPTAPE'] = np.where(demand_df3['TapeRP']==0, 0, 1)
demand_df3.fillna(0,inplace=True)


In [131]:
#adding Tape UV values 
query ="SELECT TapeId , MaterialName, IsRP FROM [Algo8].[dbo].[Master_TapeRecipe1] where MaterialName like '%uv%'"
UV_Tapes = pd.read_sql(query,engine)
uv_tapes_set = set(UV_Tapes['TapeId'])
demand_dfb =demand_df3[ demand_df3['Demand_Source']=='Buffer_demand']
demand_dfb['TapeUv'] = demand_dfb.apply(lambda row: 1 if row['TapeId'] in uv_tapes_set else 0, axis=1)


demand_df_final = pd.concat([demand_df3, demand_dfb],axis=0,ignore_index=True)
demand_df_final['tapetype'] = demand_df_final['Tape_name'].apply(lambda x: 'Webbing' if 'Webbing' in x else ('Fabrilated' if 'Fabrilated' in x else 'Other'))
demand_df_final.drop_duplicates(subset = ['TapeId','LoomNo','Tape_Target_Date'],inplace=True)
demand_df_final.to_excel("demand_final_16_01.xlsx",index=False)
#demand_df_final.to_excel("demand_final_running.xlsx",index=False)

In [114]:

demand_df_final['Tape_Target_Date'] = pd.to_datetime(demand_df_final['Tape_Target_Date']).dt.date


pivot_table = demand_df_final.pivot_table(values='Total_Demand', index='Tape_Target_Date', columns='Loom_Location', aggfunc='sum')

pivot_table.head()

Loom_Location,D-19,Gajner Road
Tape_Target_Date,,
2024-01-14,14543.55,270532.76
2024-01-15,NaN,4874.86
2024-01-16,NaN,116677.38
2024-01-17,0.00,119100.05
2024-01-18,NaN,116764.70


In [115]:
pivot_table.to_excel("day_wise_demand.xlsx")